In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 636.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17

In [2]:
!pip install --upgrade packaging torch


In [3]:
!pip install --upgrade unsloth

In [4]:
from huggingface_hub import login
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) n


In [48]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-bnb-4bit",
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2024.11.11: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [49]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj","embed_tokens", "lm_head",],
    lora_alpha = 32,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = False, # @@@ IF YOU GET OUT OF MEMORY - set to True @@@
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


In [50]:
from datasets import load_dataset

huggingface_dataset_name = "keivalya/MedQuad-MedicalQnADataset"
dataset = load_dataset(huggingface_dataset_name)

In [51]:
dataset['train'][0]

{'qtype': 'susceptibility',
 'Question': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?',
 'Answer': 'LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.'}

In [52]:
medquad_prompt = """Below is a medical question along with its context. Write a response that appropriately answers the question.

### qtype:
susceptibility

### Question:
What are the symptoms of hypertension?

### Answer:
Common symptoms of severe hypertension include headaches, difficulty breathing, and nosebleeds."""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    qtypes = examples["qtype"]        # Extract 'qtype' column
    questions = examples["Question"] # Extract 'Question' column
    answers = examples["Answer"]     # Extract 'Answer' column
    texts = []
    for qtype, question, answer in zip(qtypes, questions, answers):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = medquad_prompt.format(qtype, question, answer) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}
pass

from datasets import load_dataset
dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [55]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 4096,
    dataset_num_proc = 2,
    packing = True, # Packs short sequences together to save time!
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 1,
        learning_rate = 2e-5,
        fp16 = False,
        bf16 = False,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Generating train split: 0 examples [00:00, ? examples/s]

In [56]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 320 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 40
 "-____-"     Number of trainable parameters = 156,303,360


OutOfMemoryError: CUDA out of memory. Tried to allocate 88.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 75.06 MiB is free. Process 4124 has 14.67 GiB memory in use. Of the allocated memory 14.44 GiB is allocated by PyTorch, and 74.60 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [34]:
instruction= """You are a knowledgeable and experienced  medical assistant. Your purpose is to provide accurate, in-depth, and concise answers to patient queries related strictly to medical topics. Always maintain a polite and professional tone.
If a question is unrelated to medical topics or outside your scope of expertise, respond politely by stating: 'I’m sorry, but I can only assist with medical-related queries.
Avoid guessing or providing inaccurate information. Prioritize patient safety and reliability in your responses.
Q: Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?
A: LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents. Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.
Q: Where is Pune?
A: Sorry, I am only able to answer medical-related questions.
Q: What is recipe of Pizza?
A: Sorry, I am only able to answer medical-related questions.
"""

def format_chat_template(row):

    row_json = [{"role": "system", "content": instruction },
               {"role": "user", "content": row["Question"]},
               {"role": "assistant", "content": row["Answer"]}]

    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset["train"].map(
    format_chat_template,
    num_proc= 4,
)
dataset["text"][2]

Map (num_proc=4):   0%|          | 0/16407 [00:00<?, ? examples/s]

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [9]:
dataset = dataset.train_test_split(test_size=0.1)

In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.11.11 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field = "text",
    max_seq_length = 2048,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = True, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=4,
        eval_strategy="steps",
        eval_steps=0.2,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "model_traning_outputs",
        report_to = "none",
    ),
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

trainer_stats = trainer.train()

Map:   0%|          | 0/2979 [00:00<?, ? examples/s]

Map:   0%|          | 0/334 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,979 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 372
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss,Validation Loss
75,No log,0.835007
150,No log,0.796764
225,No log,0.779570
300,No log,0.769590


In [21]:
instruction_2 = """You are a knowledgeable and experienced  medical assistant. Your purpose is to provide accurate, in-depth, and concise answers to patient queries related strictly to medical topics. Always maintain a polite and professional tone.
If a question is unrelated to medical topics or outside your scope of expertise, respond politely by stating: 'I’m sorry, but I can only assist with medical-related queries.
Avoid guessing or providing inaccurate information. Prioritize patient safety and reliability in your responses.
Q: Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?
A: LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents. Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.
Q: Where is Pune?
A: Sorry, I am only able to answer medical-related questions.
Q: What is recipe of Pizza?
A: Sorry, I am only able to answer medical-related questions.
"""

In [28]:
FastLanguageModel.for_inference(model)

messages = [{"role": "system", "content": instruction_2},
    {"role": "user", "content": "I am having cold and fever, what treatment can I take?"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=2000, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text)

# print(text.split("assistant"))

system

Cutting Knowledge Date: December 2023
Today Date: 04 Dec 2024

You are a knowledgeable and experienced  medical assistant. Your purpose is to provide accurate, in-depth, and concise answers to patient queries related strictly to medical topics. Always maintain a polite and professional tone.
If a question is unrelated to medical topics or outside your scope of expertise, respond politely by stating: 'I’m sorry, but I can only assist with medical-related queries.
Avoid guessing or providing inaccurate information. Prioritize patient safety and reliability in your responses.
Q: Who is at risk for Lymphocytic Choriomeningitis (LCM)??
A: LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents. Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, w

In [16]:
FastLanguageModel.for_inference(model)

messages = [{"role": "system", "content": "Answer only medical-related questions. If the question is not medical, respond with 'I cannot answer this question.'"},
    {"role": "user", "content": "where is pune?"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=150, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])

# Debug: Print raw decoded output
print("Decoded Output:", text)

# Extract and print the assistant's response
response = text.split("assistant")[0] if "assistant" in text else text
print("Response:", response)



Pune is the second-most populous city in India. It is located in the state of Maharashtra, in the western part of the country. The city is situated in the Deccan Plateau region. The city is situated on the banks of the Mutha River.
Decoded Output: system

Cutting Knowledge Date: December 2023
Today Date: 03 Dec 2024

Answer only medical-related questions. If the question is not medical, respond with 'I cannot answer this question.'user

where is pune?assistant

Pune is the second-most populous city in India. It is located in the state of Maharashtra, in the western part of the country. The city is situated in the Deccan Plateau region. The city is situated on the banks of the Mutha River.
Response: system

Cutting Knowledge Date: December 2023
Today Date: 03 Dec 2024

Answer only medical-related questions. If the question is not medical, respond with 'I cannot answer this question.'user

where is pune?


In [ ]:
# Extract and print the assistant's response
response = text.split("assistant")[2] if "assistant" in text else text
print("Response:", response)

Response: 

What are the common symptoms of a cold? The common symptoms of a cold include:  Nasal congestion  Coughing  Runny nose  Sneezing  Sore throat  Fatigue (feeling tired)  Headache  Loss of appetite  Slight fever
